## 실제 추론
- 가장 최근에 Fred에 업데이트된 경제지표들로 미래에 금리 변동 예측
- 아직 업데이트 되지 않아 Nan 값인 컬럼들을 직전 기간의 데이터로 대체 

In [82]:
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
# 모델 저장
import pickle
import joblib
import datetime 
from dateutil.relativedelta import relativedelta

In [55]:
infer_df = pd.read_csv('dataframes/infer_df.csv')
total_df = pd. read_csv('dataframes/mice_selected_df.csv')

In [71]:
infer_idx = infer_df.index.start
last_total_df = total_df.tail(1)
last_total_idx = last_total_df.index.start
skip_list = ['next interest rate', 'present change']
for k in infer_df.keys():
    if k in skip_list :
        continue
    if infer_df.isna()[k][infer_idx] == True: 
        infer_df[k][infer_idx] = last_total_df[k][last_total_idx]
infer_df

,date,CPI,CPI_yoy,CPI_mom,coreCPI,coreCPI_yoy,coreCPI_mom,PCE,PCE_yoy,corePCE,corePCE_yoy,corePCE_mom,PPI,PPI_yoy,industrial production,interest rate,previous interest rate,next interest rate,previous change,present change
0,2024-09-01,1.001966,0.263807,0.544732,1.003421,0.337583,0.365682,1.0,0.34127,1.0,0.324962,0.527597,0.889652,0.348722,0.988785,5.13,5.33,NaN,-0.2,NaN


In [87]:
# best model 불러오기
model = joblib.load('best_model.pkl')
features = infer_df.loc[:, (infer_df.columns != 'date') & (infer_df.columns != 'previous interest rate') & (infer_df.columns != 'next interest rate') & (infer_df.columns != 'previous change') & (infer_df.columns != 'present change')
                  & (infer_df.columns != 'CPI')& (infer_df.columns != 'corePCE')& (infer_df.columns != 'PPI')& (infer_df.columns != 'industrial production')]
infer = model.predict(features)
infer[0]

-0.07889204557258363

In [88]:
now = datetime.datetime.today()
infer_month = (now + relativedelta(months=1)).month

file_path = 'inferece.txt' 

text = f'the day inference : {now} | target month : {infer_month}, Fed fund rate inferenced : {infer[0]}'
# 파일을 읽기 모드로 엽니다.
with open(file_path, 'r') as f:
    # 파일 내용을 읽어옵니다.
    old_text = f.read()

# 파일을 이어쓰기 모드로 엽니다.
with open(file_path, 'a') as f:
    # 수정된 내용을 파일에 이어서 씁니다.
    f.write(text)

# 파일을 닫습니다.
f.close()